<h1 align="center"><font></font>Web Scraping - Data Visualization</h1>

Membres du groupe:
- KOFFIE EZECHIEL
- KONE RAYAN
- ZAMPOU AZIZ

In [173]:
import re
import requests
import pandas as pd
import numpy as np
import seaborn as sns
from slugify import slugify
from ast import literal_eval
from bs4 import BeautifulSoup

### Partie I

In [154]:
def getPays(url):
    
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "lxml")
    country_link = soup.find("ul", {"class": "portfolio"})\
                       .findAll("li")
    base_url = "https://www.historique-meteo.net"
    liste = []
    for country in country_link:
        name = country.find("div", {"class": 'item-text'}).text
        link = country.find("a", {"class": "hover-wrap fancybox"})["href"]
        liste.append((base_url+link, name))

    return liste

In [155]:
url ="https://www.historique-meteo.net/afrique"
data = getPays(url)

In [156]:
pays_link = {}
pays_link["link"] = [t[0] for t in data]
pays_link["pays"] = [t[1] for t in data]
pays_link = pd.DataFrame(pays_link)
pays_link.to_csv("./pays_afrique.csv")

In [157]:
def getRegion(url):
    
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "lxml")
    base_url = "https://www.historique-meteo.net"
    region_link = soup.find_all("div", {"class": "list-group"},
                                recursive=True)[1].findAll("a")
    liste = []
    for region in region_link:
        name = region.find("b").text
        link = region["href"]
        liste.append((base_url+link, name))
    return liste

In [158]:
url="https://www.historique-meteo.net/afrique/cote-d-ivoire"
getRegion(url)

[('https://www.historique-meteo.net/afrique/cote-d-ivoire/abidjan/',
  'Abidjan'),
 ('https://www.historique-meteo.net/afrique/cote-d-ivoire/affery/', 'Afféry'),
 ('https://www.historique-meteo.net/afrique/cote-d-ivoire/assinie/',
  'Assinie'),
 ('https://www.historique-meteo.net/afrique/cote-d-ivoire/bondoukou/',
  'Bondoukou'),
 ('https://www.historique-meteo.net/afrique/cote-d-ivoire/bouake/', 'Bouaké'),
 ('https://www.historique-meteo.net/afrique/cote-d-ivoire/ferkessedougou/',
  'Ferkessédougou'),
 ('https://www.historique-meteo.net/afrique/cote-d-ivoire/gagnoa/', 'Gagnoa'),
 ('https://www.historique-meteo.net/afrique/cote-d-ivoire/grand-bassam/',
  'Grand-Bassam'),
 ('https://www.historique-meteo.net/afrique/cote-d-ivoire/grand-lahou/',
  'Grand-Lahou'),
 ('https://www.historique-meteo.net/afrique/cote-d-ivoire/jacqueville/',
  'Jacqueville'),
 ('https://www.historique-meteo.net/afrique/cote-d-ivoire/korhogo/',
  'Korhogo'),
 ('https://www.historique-meteo.net/afrique/cote-d-ivoi

### Partie II

In [159]:
def getDayData(url):

    req = requests.get(url)
    soup = BeautifulSoup(req.text, "lxml")
    day_data = soup.find("tbody").findAll("tr")
    label_list = []
    data_list = []
    for data in day_data:
        try : 
            label = data.find("td").text
            data_ = data.find("td", \
                    {"class": 'text-center bg-primary'}).text
        except:
            pass
        
        data_ = re.sub("[ChPa%°km/h]", "", data_)
        label_list.append(slugify(label))
        data_list.append(data_)

        
    if 'precipitations' not in label_list:
        label_list.insert(4, "precipitations")
        data_list.insert(4, np.nan)

    return label_list[:-1], data_list[:-1]

In [160]:
url="https://www.historique-meteo.net/afrique/cote-d-ivoire/abidjan/2022/01/28/"
getDayData(url)

(['temperature-maximale',
  'temperature-minimale',
  'vitesse-du-vent',
  'temperature-du-vent',
  'precipitations',
  'humidite',
  'visibilite',
  'couverture-nuageuse',
  'indice-de-chaleur',
  'point-de-rosee',
  'pression',
  'heure-du-lever-du-soleil',
  'heure-du-coucher-du-soleil',
  'duree-du-jour'],
 ['31',
  '26',
  '20',
  '26',
  nan,
  '77',
  '10',
  '17',
  '36',
  '24',
  '1012',
  '06:32:00',
  '18:25:00',
  '11:53:0'])

### Partie III

In [161]:
def getData(url, years):

    region_pays = getRegion(url)
    KPI = ['temperature-maximale', 'temperature-minimale',
           'vitesse-du-vent', 'temperature-du-vent',
           'precipitations', 'humidite', 'visibilite',
           'couverture-nuageuse', 'indice-de-chaleur',
           'point-de-rosee', 'pression', 'heure-du-lever-du-soleil',
           'heure-du-coucher-du-soleil', 'duree-du-jour']
    table = {"date": [], "region": []}
    table.update({key: [] for key in KPI})
    for year in years:
        for item in region_pays:
            for month in range(1,13):
                for day in range(1,32):
                    try:
                        day = str(day).zfill(2)
                        month = str(month).zfill(2)
                        url = item[0]+f"{year}/{month}/{day}/"
                        print(url)
                        data = getDayData(url)
                        table["date"].append(url.removeprefix(item[0]))
                        table["region"].append(item[1])
                        for kpi in data[0]:
                            table[kpi].append(data[1][data[0].index(kpi)])
                        print(table)
                    except:
                        continue

    table = pd.DataFrame(table)
    table[KPI[:-3]] = table[KPI[:-3]].astype("float")
    return table

In [ ]:
url = "https://www.historique-meteo.net/afrique/cote-d-ivoire/"
data_2022 = getData(url, [2022])

In [186]:
data_2022.to_csv("./meteo_cote_divoire.csv")

### Affichage des moyennes de chacunes des variables

In [172]:
data_2022[KPI[:-3]].apply(lambda x: np.mean(x))

temperature-maximale      32.956243
temperature-minimale      23.411050
vitesse-du-vent           12.385635
temperature-du-vent       23.411050
precipitations             5.927964
humidite                  86.176133
visibilite                 9.044254
couverture-nuageuse       51.063195
indice-de-chaleur         36.632486
point-de-rosee            22.162697
pression                1012.658785
dtype: float64

### Visualisation des données